In [1]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 

from catboost import CatBoost, Pool, CatBoostRegressor

%matplotlib inline

# Baseline

## One hot encoding

In [2]:
one_hot_data = pd.read_csv('data/one_hot_data.csv')
one_hot_data.head()

,price,year,mileage,owners_num,volume,horsepower,city kazan,city moskva,city nizhniy_novgorod,city novosibirsk,...,colour серый,colour синий,colour фиолетовый,colour чёрный,transmission механическая,transmission роботизированная,drive полный,doc_unique Оригинал,doc_unique Растаможен,engine_type Дизель
0,500000.0,2015,60500.0,2,1.6,123.0,1,0,0,0,...,0,0,0,0,1,0,0,1,0,0
1,405000.0,2012,130000.0,2,1.6,123.0,1,0,0,0,...,0,0,0,1,1,0,0,1,0,0
2,630000.0,2017,67000.0,1,1.6,123.0,1,0,0,0,...,0,0,0,0,1,0,0,1,0,0
3,620000.0,2016,80000.0,1,1.6,123.0,1,0,0,0,...,0,0,0,0,1,0,0,1,0,0
4,729000.0,2017,150000.0,3,1.6,123.0,1,0,0,0,...,0,0,0,1,1,0,0,1,0,0


In [3]:
y_one_hot = one_hot_data['price'].copy()
X_one_hot = one_hot_data.drop(['price'], axis=1)

X_train_oh, X_test_oh, y_train_oh, y_test_oh = train_test_split(X_one_hot, y_one_hot, test_size=0.3, shuffle=False)

del y_one_hot
del X_one_hot

In [4]:
%%time

lin_reg_one_hot = LinearRegression()
lin_reg_one_hot.fit(X_train_oh, y_train_oh)

print("Coef of determination:",lin_reg_one_hot.score(X_test_oh, y_test_oh))

Coef of determination: 0.8301673079196885
Wall time: 196 ms


## Label encoding

In [5]:
label_enc_data = pd.read_csv('data/label_encoded_data.csv')
label_enc_data.head(5)

,city,brand,model,price,year,mileage,body_type,colour,transmission,drive,owners_num,doc_unique,engine_type,volume,horsepower
0,1,1,3,500000.0,2015,60500.0,1,10,1,0,2,1,0,1.6,123.0
1,1,1,3,405000.0,2012,130000.0,1,14,1,0,2,1,0,1.6,123.0
2,1,1,3,630000.0,2017,67000.0,1,1,1,0,1,1,0,1.6,123.0
3,1,1,3,620000.0,2016,80000.0,1,10,1,0,1,1,0,1.6,123.0
4,1,1,3,729000.0,2017,150000.0,1,14,1,0,3,1,0,1.6,123.0


In [6]:
y_lab_enc = label_enc_data['price'].copy()
X_lab_enc = label_enc_data.drop(['price'], axis=1)

X_train_lab, X_test_lab, y_train_lab, y_test_lab = train_test_split(X_lab_enc, y_lab_enc, test_size=0.3, shuffle=False)

del y_lab_enc
del X_lab_enc

In [7]:
%%time

lin_reg_lab = LinearRegression()
lin_reg_lab.fit(X_train_lab, y_train_lab)

print("Coef of determination:", lin_reg_lab.score(X_test_lab, y_test_lab))

Coef of determination: 0.7947316862545974
Wall time: 11 ms


# Cat Boost

## Label encoding

In [8]:
%%time

model = CatBoostRegressor(iterations=100,
                           depth=2,
                           learning_rate=1,
                           loss_function='RMSE',
                         eval_metric='R2')

model.fit(X_train_lab, y_train_lab, plot=True)

predictions = model.predict(X_test_lab)
print("Coef of determination:", r2_score(y_test_lab, predictions))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.5928612	total: 69.8ms	remaining: 6.91s
1:	learn: 0.6809569	total: 72.1ms	remaining: 3.53s
2:	learn: 0.7126733	total: 73.9ms	remaining: 2.39s
3:	learn: 0.7571003	total: 76ms	remaining: 1.82s
4:	learn: 0.7767341	total: 78.1ms	remaining: 1.48s
5:	learn: 0.8053537	total: 80.4ms	remaining: 1.26s
6:	learn: 0.8222032	total: 82.5ms	remaining: 1.1s
7:	learn: 0.8303360	total: 84.8ms	remaining: 975ms
8:	learn: 0.8395073	total: 87.4ms	remaining: 883ms
9:	learn: 0.8467978	total: 90.3ms	remaining: 813ms
10:	learn: 0.8540234	total: 92.9ms	remaining: 752ms
11:	learn: 0.8603358	total: 94.9ms	remaining: 696ms
12:	learn: 0.8640888	total: 97.2ms	remaining: 650ms
13:	learn: 0.8676544	total: 99.2ms	remaining: 609ms
14:	learn: 0.8705118	total: 101ms	remaining: 573ms
15:	learn: 0.8738628	total: 103ms	remaining: 542ms
16:	learn: 0.8771262	total: 105ms	remaining: 514ms
17:	learn: 0.8794486	total: 108ms	remaining: 491ms
18:	learn: 0.8819489	total: 110ms	remaining: 469ms
19:	learn: 0.8833945	total: 11

## One hot encoding

In [9]:
%%time

model = CatBoostRegressor(iterations=100,
                           depth=2,
                           learning_rate=1,
                           loss_function='RMSE', 
                         eval_metric='R2')

model.fit(X_train_oh, y_train_oh, plot=True)

predictions = model.predict(X_test_oh)
print("Coef of determination:", r2_score(y_test_oh, predictions))

0:	learn: 0.5923791	total: 6.12ms	remaining: 606ms


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

1:	learn: 0.6846898	total: 12.2ms	remaining: 600ms
2:	learn: 0.7254459	total: 14.8ms	remaining: 478ms
3:	learn: 0.7487442	total: 17.4ms	remaining: 417ms
4:	learn: 0.7750667	total: 19.7ms	remaining: 375ms
5:	learn: 0.7992448	total: 22ms	remaining: 345ms
6:	learn: 0.8085816	total: 27.3ms	remaining: 363ms
7:	learn: 0.8237166	total: 29.8ms	remaining: 343ms
8:	learn: 0.8340972	total: 32.4ms	remaining: 328ms
9:	learn: 0.8413904	total: 34.8ms	remaining: 313ms
10:	learn: 0.8468623	total: 36.9ms	remaining: 299ms
11:	learn: 0.8515202	total: 39.3ms	remaining: 288ms
12:	learn: 0.8571011	total: 41.8ms	remaining: 280ms
13:	learn: 0.8613704	total: 43.9ms	remaining: 270ms
14:	learn: 0.8654196	total: 46.5ms	remaining: 264ms
15:	learn: 0.8678626	total: 48.7ms	remaining: 256ms
16:	learn: 0.8739701	total: 51ms	remaining: 249ms
17:	learn: 0.8779511	total: 53.9ms	remaining: 245ms
18:	learn: 0.8821506	total: 56.4ms	remaining: 240ms
19:	learn: 0.8844606	total: 59.1ms	remaining: 236ms
20:	learn: 0.8861815	tota

# Chose models with good price

In [10]:
X = label_enc_data.drop(['price'], axis=1)
y = label_enc_data['price'].copy()

model = CatBoostRegressor(iterations=100,
                           depth=2,
                           learning_rate=1,
                           loss_function='RMSE')

model.fit(X, y)

predictions = model.predict(X)

0:	learn: 133836.4535310	total: 3.84ms	remaining: 380ms
1:	learn: 116699.1894795	total: 7.17ms	remaining: 351ms
2:	learn: 106181.9449914	total: 10.9ms	remaining: 351ms
3:	learn: 100916.9313588	total: 13.4ms	remaining: 322ms
4:	learn: 96994.3818949	total: 15.9ms	remaining: 302ms
5:	learn: 90504.4630451	total: 18.4ms	remaining: 288ms
6:	learn: 88221.9889111	total: 20.9ms	remaining: 277ms
7:	learn: 86836.7373666	total: 24.1ms	remaining: 277ms
8:	learn: 83578.5006895	total: 27ms	remaining: 273ms
9:	learn: 82326.7492165	total: 29.3ms	remaining: 263ms
10:	learn: 80982.6531346	total: 31.6ms	remaining: 256ms
11:	learn: 79521.4866257	total: 34ms	remaining: 250ms
12:	learn: 77163.2737813	total: 36.5ms	remaining: 245ms
13:	learn: 75330.0413721	total: 38.9ms	remaining: 239ms
14:	learn: 74172.0352259	total: 41.3ms	remaining: 234ms
15:	learn: 73410.5618353	total: 43.6ms	remaining: 229ms
16:	learn: 72578.3203546	total: 46.3ms	remaining: 226ms
17:	learn: 71992.0527479	total: 49ms	remaining: 223ms
18:	

In [12]:
def good_price_or_not(real, predicted):
    if predicted > real:
        return 'Good price'
    else:
        return 'Bad price'

In [13]:
for real, pred in zip(y, predictions):
    print(real, pred, good_price_or_not(real, pred))

500000.0 551525.9393464591 Good price
405000.0 441896.72211433284 Good price
630000.0 677973.9542475565 Good price
620000.0 607271.5598566723 Bad price
729000.0 586083.8329293766 Bad price
720000.0 691861.4029622187 Bad price
730000.0 682013.6627906536 Bad price
790000.0 758227.5301567616 Bad price
680000.0 698711.1431907278 Good price
760000.0 808138.1876811297 Good price
720000.0 777082.8337979713 Good price
770000.0 864601.3888019623 Good price
620000.0 616312.7918693695 Bad price
620000.0 677973.9542475565 Good price
750000.0 739853.459169626 Bad price
509000.0 434741.92858096596 Bad price
870000.0 682013.6627906536 Bad price
629500.0 697917.6714453155 Good price
785000.0 782611.7278989265 Bad price
770000.0 799894.4634384366 Good price
830000.0 746031.2968499508 Bad price
420000.0 409501.85225735145 Bad price
680000.0 745237.8251045384 Good price
379000.0 408884.09098471445 Good price
815000.0 908350.8914851035 Good price
780000.0 805678.7022254622 Good price
780000.0 810119.66639

525000.0 576949.4533454054 Good price
450000.0 432299.45872439654 Bad price
635000.0 597391.7815914899 Bad price
650000.0 696113.1399344563 Good price
360000.0 373106.1087116699 Good price
570000.0 597882.8662742315 Good price
430000.0 432960.2663853308 Good price
389000.0 435989.33734455233 Good price
620000.0 617264.1165333226 Bad price
410000.0 356612.88141694973 Bad price
530000.0 438491.2157124808 Bad price
335000.0 334492.39554576005 Bad price
290000.0 298749.51780821016 Good price
610000.0 691932.3805342388 Good price
415000.0 390275.7763117687 Bad price
550000.0 526961.1139146434 Bad price
600000.0 466573.2512061898 Bad price
675000.0 619227.7364318693 Bad price
400000.0 369147.6373258659 Bad price
738000.0 800235.0487381897 Good price
580000.0 630543.2701603386 Good price
600000.0 664025.6375833866 Good price
495000.0 483429.85611859744 Bad price
590000.0 617307.4282151288 Good price
865000.0 676551.6115391718 Bad price
379000.0 376862.1839963151 Bad price
730000.0 722326.0494

390000.0 353665.0697734712 Bad price
379000.0 412927.7148827235 Good price
430000.0 516777.41380547715 Good price
700000.0 711888.159270285 Good price
445000.0 473598.1256133672 Good price
449000.0 446145.8855985166 Bad price
480000.0 448215.4164452239 Bad price
675000.0 677739.6235296348 Good price
650000.0 715762.4984545694 Good price
340000.0 373656.84274356626 Good price
300000.0 352659.71894698055 Good price
630000.0 645614.5284846509 Good price
840000.0 901104.1662927377 Good price
475000.0 502972.96665272623 Good price
435000.0 475397.7327705816 Good price
410000.0 443828.13371566276 Good price
470000.0 428675.4446702256 Bad price
628000.0 710029.473468066 Good price
285000.0 246126.5627981173 Bad price
190000.0 130276.31787789735 Bad price
320000.0 283006.8219215582 Bad price
255000.0 325284.54489691125 Good price
450000.0 420238.03921152593 Bad price
580000.0 612962.0293165038 Good price
750000.0 789766.7025523835 Good price
790000.0 697070.4332479588 Bad price
180000.0 181536

550000.0 469281.690051634 Bad price
520000.0 578867.3550941296 Good price
180000.0 140054.62689016474 Bad price
720000.0 699745.1275976503 Bad price
710000.0 756382.2285615536 Good price
750000.0 757830.0341060112 Good price
695000.0 720296.1244168908 Good price
600000.0 618615.1691110898 Good price
700000.0 661009.5928072094 Bad price
420000.0 425976.05574319843 Good price
700000.0 735113.6509221973 Good price
750000.0 705966.0583546382 Bad price
175000.0 219918.47632892418 Good price
730000.0 660950.8164752664 Bad price
585000.0 545713.1676974 Bad price
780000.0 721985.8482624616 Bad price
395000.0 424237.1330283928 Good price
725000.0 715855.1602499682 Bad price
470000.0 556703.2780007267 Good price
430000.0 431776.2997473059 Good price
550000.0 546736.4145556298 Bad price
535000.0 547795.6650462522 Good price
430000.0 469345.2991276369 Good price
710000.0 721192.3765170493 Good price
425000.0 433126.17998133396 Good price
769000.0 824004.7097842321 Good price
850000.0 769179.419967

480000.0 453546.134348743 Bad price
565000.0 596706.2076544927 Good price
609990.0 617580.5718000717 Good price
1010000.0 1141674.7933538668 Good price
630000.0 703049.4458656335 Good price
420000.0 479495.7867256388 Good price
630000.0 654707.3167479574 Good price
720000.0 726247.0484971158 Good price
520000.0 546733.9691533436 Good price
500000.0 515379.4934987329 Good price
710000.0 747801.237845079 Good price
730000.0 692281.1020753791 Bad price
640000.0 654707.3167479574 Good price
429000.0 493066.9998388837 Good price
470000.0 552656.5862372778 Good price
610000.0 694765.8464888599 Good price
449000.0 536461.2273962081 Good price
615000.0 609682.4131733372 Bad price
380000.0 508871.0799982839 Good price
400000.0 452253.0885221698 Good price
385000.0 409206.9130298658 Good price
430000.0 474157.83214987576 Good price
650000.0 629666.7932548391 Bad price
400000.0 453154.56751306355 Good price
705000.0 624374.8179062926 Bad price
480000.0 476458.0022514871 Bad price
660000.0 630159.

...

450000.0 437242.099028606 Bad price
470000.0 488427.54058889416 Good price
495000.0 498105.10005364806 Good price
839000.0 820140.1725157187 Bad price
650000.0 581326.8621005204 Bad price
618000.0 601055.486276408 Bad price
490000.0 544548.8306931334 Good price
579000.0 564965.9699736724 Bad price
659000.0 595286.3035892035 Bad price
427000.0 423538.77358546644 Bad price
469000.0 443552.0871871477 Bad price
405000.0 397477.9348923684 Bad price
465000.0 430765.47651741887 Bad price
739000.0 699461.9857255036 Bad price
510000.0 515516.8238795496 Good price
639000.0 585869.2455042427 Bad price
400000.0 435764.89655129064 Good price
769000.0 751123.944424151 Bad price
550000.0 529477.6383472665 Bad price
390000.0 409361.8424531128 Good price
679000.0 630998.155629624 Bad price
500000.0 489695.7582750833 Bad price
439000.0 406450.90845612925 Bad price
450000.0 429456.5079296136 Bad price
419000.0 418918.18630287994 Bad price
300000.0 505170.37661217735 Good price
465000.0 479961.5883584884 

385000.0 370547.1129924656 Bad price
398000.0 364148.04491875356 Bad price
595000.0 616926.5576250452 Good price
699000.0 691041.8508286361 Bad price
484000.0 479347.09680085594 Bad price
519000.0 539655.1308174775 Good price
475000.0 467330.8989566715 Bad price
400000.0 452175.4077829968 Good price
285000.0 348321.73141377314 Good price
65000.0 89032.23952442384 Good price
470000.0 446933.4303277702 Bad price
700000.0 618157.8464647024 Bad price
510000.0 472055.5317090626 Bad price
569000.0 581761.6157135514 Good price
400000.0 474458.14830500504 Good price
400000.0 447338.299986126 Good price
449000.0 479819.2045788835 Good price
590000.0 565516.0545712714 Bad price
486900.0 469530.39438243734 Bad price
470000.0 521543.63617421995 Good price
584000.0 635688.2874450162 Good price
400000.0 385126.9655904558 Bad price
467000.0 488387.0363834374 Good price
399000.0 382846.34899330355 Bad price
295000.0 376369.2688911424 Good price
740000.0 726742.3250077592 Bad price
249000.0 334783.6978

699000.0 695429.6319478534 Bad price
825000.0 835690.2912254846 Good price
650000.0 708471.7266056461 Good price
660000.0 706287.1519338461 Good price
385000.0 411987.55128331645 Good price
425000.0 452480.12722998654 Good price
700000.0 771187.9048347923 Good price
430000.0 431349.150730779 Good price
490000.0 517689.57209383626 Good price
720000.0 754708.476722557 Good price
550000.0 540377.4490021935 Bad price
510000.0 450902.42070171796 Bad price
399000.0 396214.69933197356 Bad price
650000.0 660523.0090014015 Good price
739000.0 709988.9832106997 Bad price
636000.0 621179.5900158312 Bad price
155000.0 134288.48312197032 Bad price
330000.0 349499.76829713205 Good price
400000.0 392014.29527255817 Bad price
459000.0 451402.8059249414 Bad price
395000.0 369823.1491393609 Bad price
535000.0 506542.8753920136 Bad price
925000.0 908310.6513575749 Bad price
665000.0 732870.1166498978 Good price
128000.0 175870.78425493737 Good price
870000.0 900268.887629881 Good price
150000.0 189014.70

550000.0 641692.796177934 Good price
515000.0 545352.5309738897 Good price
875000.0 813930.1188296897 Bad price
595000.0 604909.0140026651 Good price
545000.0 636823.0486538399 Good price
340000.0 406656.4696071284 Good price
350000.0 376872.8643831846 Good price
480000.0 425591.45483254094 Bad price
630000.0 642372.2318099813 Good price
580000.0 626680.9855239808 Good price
340000.0 354138.08388221543 Good price
420000.0 405524.58338065015 Bad price
525000.0 599621.7080426872 Good price
395000.0 371432.7513636917 Bad price
650000.0 621709.2592558981 Bad price
380000.0 540231.5729565306 Good price
645000.0 636170.906385374 Bad price
710000.0 751316.2451720102 Good price
1050000.0 1058093.2362409036 Good price
329999.0 320991.77591195883 Bad price
550000.0 539298.0223970072 Bad price
335000.0 390343.0819338795 Good price
660000.0 658803.6874460897 Bad price
465000.0 460476.4845522764 Bad price
425000.0 396628.4820735471 Bad price
675000.0 742370.1055032627 Good price
649990.0 648193.631

795000.0 749819.0305263427 Bad price
360000.0 453379.00599424983 Good price
330000.0 378500.8640838818 Good price
461000.0 438711.876214005 Bad price
600000.0 565711.7065493285 Bad price
389000.0 391284.6065606426 Good price
425000.0 445539.2439993556 Good price
450000.0 388445.0050227964 Bad price
760000.0 822535.5615222564 Good price
400000.0 396121.902489631 Bad price
670000.0 674474.0821090196 Good price
780000.0 800819.1251688765 Good price
620000.0 621026.0084824591 Good price
489000.0 507089.78185417136 Good price
430000.0 392310.235745716 Bad price
360000.0 365499.5497591423 Good price
465000.0 492283.83900206134 Good price
420000.0 415221.8771665192 Bad price
350000.0 382219.9991882916 Good price
360000.0 398322.0716488085 Good price
479000.0 587211.1751224917 Good price
670000.0 657615.993257704 Bad price
600000.0 655554.9213843792 Good price
430000.0 431155.8121562539 Good price
500000.0 434986.2328879031 Bad price
50000.0 102517.32731681864 Good price
700000.0 616738.398258